## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-08-10-52-07 +0000,bbc,At least 13 dead after youth protests against ...,https://www.bbc.com/news/articles/c78nd2zy9jgo...
1,2025-09-08-10-43-32 +0000,nyt,Jeffrey Epstein’s Bankers,https://www.nytimes.com/2025/09/08/world/jeffr...
2,2025-09-08-10-30-00 +0000,wapo,It’s not just Trump. Red states are cracking d...,https://www.washingtonpost.com/nation/2025/09/...
3,2025-09-08-10-29-47 +0000,bbc,'It's getting harder and harder to run a pub',https://www.bbc.com/news/articles/crme2jr3j2vo...
4,2025-09-08-10-27-11 +0000,nyt,Oil Prices Rise After Producers Announce New I...,https://www.nytimes.com/2025/09/07/business/op...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,21
229,raid,11
309,south,10
30,new,10
23,run,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
233,2025-09-07-12-19-07 +0000,nypost,South Korean workers arrested in ICE raid at G...,https://nypost.com/2025/09/07/us-news/south-ko...,96
215,2025-09-07-15-08-17 +0000,nypost,Russian attack on Ukraine’s government HQ in l...,https://nypost.com/2025/09/07/world-news/russi...,71
109,2025-09-08-00-12-10 +0000,wapo,South Korea announces deal to repatriate worke...,https://www.washingtonpost.com/world/2025/09/0...,64
133,2025-09-07-22-34-59 +0000,latimes,"South Korea, irked at U.S. raid at Hyundai pla...",https://www.latimes.com/business/story/2025-09...,64
214,2025-09-07-15-11-59 +0000,nyt,South Korea Negotiates Release of Korean Worke...,https://www.nytimes.com/2025/09/07/world/asia/...,61


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
233,96,2025-09-07-12-19-07 +0000,nypost,South Korean workers arrested in ICE raid at G...,https://nypost.com/2025/09/07/us-news/south-ko...
182,53,2025-09-07-17-38-00 +0000,wsj,Russia’s Largest Drone Attack Yet Hits Ukraine...,https://www.wsj.com/world/russias-largest-dron...
83,41,2025-09-08-03-34-58 +0000,nypost,Father on run with 3 kids in wilderness for 4 ...,https://nypost.com/2025/09/07/world-news/fugit...
108,36,2025-09-08-00-22-05 +0000,nyt,"Australian Woman, Erin Patterson, Gets Life in...",https://www.nytimes.com/2025/09/07/world/austr...
232,36,2025-09-07-12-25-45 +0000,nyt,Prime Minister Shigeru Ishiba of Japan Says He...,https://www.nytimes.com/2025/09/07/world/asia/...
10,29,2025-09-08-10-00-00 +0000,latimes,California FAIR Plan continues denying smoke d...,https://www.latimes.com/business/story/2025-09...
0,28,2025-09-08-10-52-07 +0000,bbc,At least 13 dead after youth protests against ...,https://www.bbc.com/news/articles/c78nd2zy9jgo...
27,26,2025-09-08-09-30-00 +0000,wsj,The U.S. labor market would be barely growing ...,https://www.wsj.com/economy/jobs/healthcare-jo...
207,26,2025-09-07-16-00-08 +0000,cbc,Rare hit by Yemen's Houthi rebels strikes Isra...,https://www.cbc.ca/news/world/houthi-drone-str...
120,26,2025-09-07-23-33-59 +0000,latimes,"City buys Griffith Park carousel, where Disney...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
